# Data Preparation

## Packages

In [1]:
import os
import winsound
import numpy as np
import pandas as pd
import datetime as dt
from tqdm import tqdm
tqdm.pandas()
from multiprocessing import  Pool
import time

import math, collections
from scipy.stats import linregress

from matplotlib import pyplot as plt
from matplotlib.pyplot import figure

import warnings
warnings.filterwarnings('ignore')

In [2]:
def get_slope(y_axis):
    global data
    ma_len = len(y_axis)
    
    x_axis = []
    for i in range(ma_len):
        x_axis.append(1 + ((i+1) * 0.0001 * 0.1))
    
    slope_tick, intercept, _, _, _ = linregress(x_axis, y_axis)
    slope_tick = math.degrees(math.atan(slope_tick))
    
    return(slope_tick)

In [3]:
def chunk_ticks(df, number_of_ticks):   
    global data
    
    df['tick'] = (df['Bid'] + df['Ask'])/2
    df['spread'] = df['Ask'] - df['Bid']
    df = df[['tick', 'spread']]
    
    temp_df = pd.DataFrame()
    tick_avg = []
    spread_avg = []
    tick_sd = []
    
    for i in tqdm(range(0,len(df),number_of_ticks)):
        tick_list = list(df['tick'][i:i+number_of_ticks])
        spread_list = list(df['spread'][i:i+number_of_ticks])
        
        tick_avg.append(np.mean(tick_list))
        spread_avg.append(np.mean(spread_list))
        tick_sd.append(np.std(tick_list))
        
        
    temp_df['tick_avg'] = tick_avg  
    temp_df['spread_avg'] = spread_avg  
    temp_df['tick_sd'] = tick_sd  
    
    return(temp_df)

In [4]:
def before_sma():
    global data    
    data['ssma_list'].append(val)    
    return()

def after_sma():
    global data
    
    data['ssma_list'].popleft()
    data['ssma_list'].append(val)
    data['sema'] = list(pd.DataFrame(list(data['ssma_list'])).ewm(span=data['sma_len']).mean()[0])[data['sma_len'] - 1]
    
    if len(data['sema_ready']) < 2:
        data['sema_ready'].append(data['sema'])
        data['sema_diff'] = np.nan

    elif len(data['sema_ready']) > 1:
        data['sema_ready'].popleft()
        data['sema_ready'].append(data['sema'])
        data['sema_diff'] = data['sema_ready'][-1] - data['sema_ready'][len(data['sema_ready'])-2]
    
    return()

def before_lma():
    global data    
    data['lsma_list'].append(val)    
    return()

def after_lma():
    global data
    
    data['lsma_list'].popleft()
    data['lsma_list'].append(val)
    data['lema'] = list(pd.DataFrame(list(data['lsma_list'])).ewm(span=data['lma_len']).mean()[0])[data['lma_len'] - 1]
    
    if len(data['lema_ready']) < 2:
        data['lema_ready'].append(data['lema'])
        data['lema_diff'] = np.nan

    elif len(data['lema_ready']) > 1:
        data['lema_ready'].popleft()
        data['lema_ready'].append(data['lema'])
        data['lema_diff'] = data['lema_ready'][-1] - data['lema_ready'][len(data['lema_ready'])-2]
    
    return()

In [5]:
def roll_ma(ma_list):
    global data
    ma_len = len(ma_list)
    sema_val = list(pd.DataFrame(ma_list).ewm(span=ma_len).mean()[0])[ma_len - 1]    
    return(sema_val)

## File paths

#### 1. High number of ticks ensures pip level movement and pip level predictions
#### 2. High sma_len helps ml model prediction accuracy (Predictable curve)
#### 3. High sma_len remove connection between actual tick_avg and sema (Prediction is high, but actual prediction is poor)

In [6]:
year = 2019

data = {}
data['number_of_ticks'] = 5
data['rsi_window'] = 14
data['sma_len'] = 20
data['lma_len'] = 40

diff_col = 'sema'
data['pip_diff'] = 0.0001

source_file_path = f'data/tick_{year}.csv'
path, file_name = os.path.split(source_file_path)

target_file_name = 'tab_'+file_name
target_file_path = os.path.join(path, target_file_name)

chunk_file_name = 'chunk_'+file_name
chunk_file_path = os.path.join(path, chunk_file_name)

print(f'source_file_path : {source_file_path}')
print(f'chunk_file_path : {chunk_file_path}')
print(f'target_file_path : {target_file_path}')

source_file_path : data/tick_2019.csv
chunk_file_path : data\chunk_tick_2019.csv
target_file_path : data\tab_tick_2019.csv


## Read data

In [7]:
%%time
df = pd.read_csv(source_file_path, nrows=1000000)
#df = pd.read_csv(source_file_path)
df.head()

Wall time: 537 ms


,DateTime,Bid,Ask,Volume
0,20190101 22:02:37.254,1.14598,1.14682,4
1,20190101 22:02:38.590,1.14599,1.14682,2
2,20190101 22:02:39.138,1.14599,1.14684,4
3,20190101 22:02:55.787,1.14598,1.14684,4
4,20190101 22:03:02.060,1.14598,1.14684,4


## Data manipulation

In [8]:
%%time

df = chunk_ticks(df, data['number_of_ticks'])
df.to_csv(chunk_file_path, index = False)
print(f'Records : {len(df)}')

df = pd.read_csv(chunk_file_path)
df.head()

100%|███████████████████████████████████████████████████████| 200000/200000 [00:44<00:00, 4487.55it/s]


Records : 200000
Wall time: 46.3 s


,tick_avg,spread_avg,tick_sd
0,1.146408,0.000848,0.000005
1,1.146440,0.000804,0.000029
2,1.146382,0.000628,0.000037
3,1.146384,0.000648,0.000029
4,1.146500,0.000708,0.000044


In [9]:
%%time

data['rs_max'] = 1e6

data['ssma_list'] = collections.deque([])
data['lsma_list'] = collections.deque([])
data['sema_ready'] = collections.deque([])
data['lema_ready'] = collections.deque([])
df['sema'] = ''
df['lema'] = ''
df['sema_diff'] = ''
df['lema_diff'] = ''


# RSI -----------------------------
df['diff'] = df['tick_avg'].diff()
df['gain'] = 0
df['loss'] = 0
df['gain'].loc[df['diff'] > 0] = abs(df['diff'])
df['loss'].loc[df['diff'] < 0] = abs(df['diff'])
df['avg_gain'] = df['gain'].rolling(window=data['rsi_window']).mean()
df['avg_loss'] = df['loss'].rolling(window=data['rsi_window']).mean()
df['rs'] = df['avg_gain']/df['avg_loss']
df['rs'] = df['rs'].where(df['rs'] <= data['rs_max'], data['rs_max']) 
df['rsi'] = 100 - (100 / (df['rs'] + 1))

# Simple Moving Averages ------------------
df['ssma'] = df['tick_avg'].rolling(window=data['sma_len']).mean()
df['ssma_diff'] = df['ssma'].diff()
df['lsma'] = df['tick_avg'].rolling(window=data['lma_len']).mean()
df['lsma_diff'] = df['lsma'].diff()
df['sma_diff'] = df['ssma'] - df['lsma']

df['max_tick'] = df['tick_avg'].rolling(window=data['sma_len']).max()
df['min_tick'] = df['tick_avg'].rolling(window=data['sma_len']).min()

df['max_gap'] = df['max_tick'] -  df['tick_avg']
df['min_gap'] = df['min_tick'] - df['tick_avg']

Wall time: 167 ms


In [10]:
%%time
# Emas ----------------
df['sema'] = df['tick_avg'].rolling(window=data['sma_len']).progress_apply(roll_ma)
df['lema'] = df['tick_avg'].rolling(window=data['lma_len']).progress_apply(roll_ma)

df['sema_diff'] = df['sema'].diff()
df['lema_diff'] = df['lema'].diff()

199981it [03:05, 1076.67it/s]
199961it [03:09, 1056.51it/s]

Wall time: 6min 15s


In [11]:
%%time
# Slopes -----------------------------
df['small_sema_slope'] = df['sema'].rolling(window=data['sma_len']).progress_apply(get_slope)
df['long_sema_slope'] = df['sema'].rolling(window=data['lma_len']).progress_apply(get_slope)

199962it [00:57, 3452.06it/s]
199942it [00:58, 3416.42it/s]

Wall time: 1min 56s


In [12]:
%%time

df['ema_diff'] = df['sema'] - df['lema']

# Direction -------------------------
df['direction'] = 'same'
df['direction'].loc[df[diff_col].shift(-1) - df[diff_col] > data['pip_diff']] = 'increase'
df['direction'].loc[df[diff_col].shift(-1) - df[diff_col] < -data['pip_diff']] = 'decrease'

# Remove NaNs ------------------------
del df['gain']
del df['loss']
df = df.dropna()
df = df.reset_index(drop=True)
print(f'Total records : {len(df)}')

df.tail()

Total records : 199942
Wall time: 172 ms


,tick_avg,spread_avg,tick_sd,sema,lema,sema_diff,lema_diff,diff,avg_gain,avg_loss,...,lsma_diff,sma_diff,max_tick,min_tick,max_gap,min_gap,small_sema_slope,long_sema_slope,ema_diff,direction
199937,1.152378,0.000028,0.000009,1.152382,1.152401,-0.000001,-1.685163e-06,0.000018,0.000013,0.000012,...,-0.000002,-0.000027,1.152432,1.152332,0.000054,-0.000046,-15.859396,-7.607843,-0.000019,same
199938,1.152397,0.000038,0.000004,1.152383,1.152400,0.000001,-5.235040e-07,0.000019,0.000014,0.000012,...,-0.000001,-0.000027,1.152432,1.152332,0.000035,-0.000065,-15.519710,-8.501874,-0.000018,same
199939,1.152410,0.000040,0.000007,1.152385,1.152400,0.000002,8.275987e-08,0.000013,0.000015,0.000012,...,-0.000001,-0.000026,1.152432,1.152332,0.000022,-0.000078,-14.743992,-9.224343,-0.000015,same
199940,1.152382,0.000036,0.000007,1.152384,1.152399,-0.000001,-1.645803e-06,-0.000028,0.000011,0.000014,...,-0.000002,-0.000026,1.152432,1.152332,0.000050,-0.000050,-13.732619,-9.791717,-0.000015,same
199941,1.152371,0.000038,0.000008,1.152382,1.152397,-0.000002,-2.224207e-06,-0.000011,0.000011,0.000012,...,-0.000003,-0.000027,1.152432,1.152332,0.000061,-0.000039,-12.556386,-10.221293,-0.000015,same


## Write data to csv

In [13]:
%%time
df.to_csv(target_file_path, index = False)
winsound.PlaySound('C:\\Windows\\Media\\tada.wav', winsound.SND_ASYNC)

Wall time: 6.95 s


## Print Report

In [14]:
g= df['direction']
print(pd.concat([g.value_counts(), g.value_counts(normalize=True).mul(100)],axis=1, keys=('counts','percentage')))
df.head(5)

          counts  percentage
same      199918   99.987997
decrease      19    0.009503
increase       5    0.002501


,tick_avg,spread_avg,tick_sd,sema,lema,sema_diff,lema_diff,diff,avg_gain,avg_loss,...,lsma_diff,sma_diff,max_tick,min_tick,max_gap,min_gap,small_sema_slope,long_sema_slope,ema_diff,direction
0,1.146262,0.000376,0.000002,1.146331,1.146386,-0.000009,-0.000010,1.000000e-06,0.000015,0.000025,...,-0.000010,-0.000105,1.146413,1.146218,0.000151,-0.000044,-39.176807,-45.164546,-0.000056,same
1,1.146259,0.000378,0.000002,1.146322,1.146377,-0.000009,-0.000009,-3.000000e-06,0.000014,0.000025,...,-0.000010,-0.000102,1.146413,1.146218,0.000154,-0.000041,-37.476936,-45.850329,-0.000055,same
2,1.146260,0.000372,0.000003,1.146314,1.146368,-0.000008,-0.000009,1.000000e-06,0.000014,0.000024,...,-0.000010,-0.000098,1.146413,1.146218,0.000153,-0.000042,-35.936688,-46.338119,-0.000054,same
3,1.146259,0.000374,0.000004,1.146307,1.146360,-0.000008,-0.000009,-1.000000e-06,0.000014,0.000011,...,-0.000010,-0.000095,1.146413,1.146218,0.000154,-0.000041,-34.421751,-46.640898,-0.000053,same
4,1.146258,0.000372,0.000002,1.146300,1.146351,-0.000007,-0.000008,-1.000000e-06,0.000014,0.000012,...,-0.000011,-0.000093,1.146413,1.146218,0.000155,-0.000040,-32.862303,-46.740553,-0.000052,same


In [15]:
# Direction -------------------------
df['act_direction'] = 'same'
df['act_direction'].loc[df['tick_avg'].shift(-1) - df['tick_avg'] > 0.0002] = 'increase'
df['act_direction'].loc[df['tick_avg'].shift(-1) - df['tick_avg'] < -0.0002] = 'decrease'

print('prediction : same')
print(df.loc[df['direction'] == 'same', 'act_direction'].value_counts(normalize=True))
print('-------------')

print('prediction : increase')
print(df.loc[df['direction'] == 'increase', 'act_direction'].value_counts(normalize=True))
print('-------------')

print('prediction : decrease')
print(df.loc[df['direction'] == 'decrease', 'act_direction'].value_counts(normalize=True))
print('-------------')

print('\n')
print(df[['tick_avg', 'sema', 'direction','act_direction']].head(50))
del df['act_direction']

prediction : same
same        0.998915
increase    0.000575
decrease    0.000510
Name: act_direction, dtype: float64
-------------
prediction : increase
increase    0.6
same        0.4
Name: act_direction, dtype: float64
-------------
prediction : decrease
same        0.631579
decrease    0.315789
increase    0.052632
Name: act_direction, dtype: float64
-------------


    tick_avg      sema direction act_direction
0   1.146262  1.146331      same          same
1   1.146259  1.146322      same          same
2   1.146260  1.146314      same          same
3   1.146259  1.146307      same          same
4   1.146258  1.146300      same          same
5   1.146260  1.146294      same          same
6   1.146258  1.146288      same          same
7   1.146258  1.146283      same          same
8   1.146267  1.146280      same          same
9   1.146275  1.146280      same          same
10  1.146257  1.146278      same          same
11  1.146324  1.146284      same          same
12  1.146292  1.1